# Env Setup

In [1]:
!pip install -U "ray[rllib]==1.11.0"

     |████████████████████████████████| 52.7 MB 160 kB/s 
     |████████████████████████████████| 4.1 MB 43.3 MB/s 
     |████████████████████████████████| 226 kB 51.0 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 125 kB 38.8 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [2]:
# Visualizer setup
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


In [3]:
import gym
import torch

import numpy as np
from collections import namedtuple
import random, math
import matplotlib.pyplot as plt
from ray.rllib.models import ModelCatalog

from ray.rllib.env.env_context import EnvContext
import time
import os

from gym.spaces import Discrete, Box
from ray import tune
import ray
import ray.rllib.agents.ppo as ppo
import ray.rllib.agents.sac as sac

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Visualization

In [5]:
# How to get gym AI visuals in colab
# https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb#scrollTo=T9RpF49oOsZj

from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Test with random actions 

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

game =  "CartPole-v1"
env =  wrap_env(gym.make(game))

print("game: ", game)
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

print("States size:", num_states, "Actions size: ",  num_actions)

device:  cuda:0
game:  CartPole-v1
States size: 4 Actions size:  2


In [7]:
done = False

env.reset()

while not done:
    env.render()
    state, reward, done, _ = env.step(env.action_space.sample()) # take a random action


env.close()
show_video()

# Use ray tune for hyperparameter search 

In [8]:
import json
def evaluation_fn(result):
    return result['episode_reward_mean']


def objective_fn(config, checkpoint_dir = "/content/drive/MyDrive/INM707/cartpole_v1"):
    start = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            start = state["step"] + 1


    trainer = sac.SACTrainer(config = config) 

    for i in range(start, 10):
      # Perform one iteration of training the policy with DQN
      result = trainer.train()
      intermediate_score = evaluation_fn(result)

      with tune.checkpoint_dir(step=i) as checkpoint_dir:
          path = os.path.join(checkpoint_dir, "checkpoint")
          with open(path, "w") as f:
              f.write(json.dumps({"step": i}))
    
      # Feed the score back back to Tune.
      tune.report(iterations=i, mean_reward=intermediate_score)

## config

In [9]:
ray.shutdown()
ray.init(num_cpus= 2, num_gpus=1)

{'address': '172.28.0.2:63104',
 'gcs_address': '172.28.0.2:63104',
 'metrics_export_port': 58045,
 'node_id': '655ed2088c7835f9b34efb4a447867ad44969bcca3ed4c4aa9df2069',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-04-22_11-19-54_557197_72/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-04-22_11-19-54_557197_72/sockets/raylet',
 'redis_address': None,
 'session_dir': '/tmp/ray/session_2022-04-22_11-19-54_557197_72',
 'webui_url': None}

In [10]:
config = sac.DEFAULT_CONFIG.copy()

config["framework"] = "torch"
config["train_batch_size"] = tune.grid_search([32, 64])
config["target_network_update_freq"] = 32 #tune.grid_search([16, 32])
config["env"] = 'CartPole-v0' 
config["gamma"] = 0.95 #tune.uniform(0, 1)

#used from tuned values : https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/sac/cartpole-sac.yaml
config["tau"] = 1
config["optimization"] = {
        "actor_learning_rate": 0.005,
        "critic_learning_rate": 0.005,
        "entropy_learning_rate": 0.0001}
config["no_done_at_end"] = False


config["num_gpus"] = 1

In [11]:
analysis = tune.run(
        objective_fn,
        stop={"mean_reward": 200},
        metric="mean_reward",
        mode="max",
        resources_per_trial={'gpu': 1},
        num_samples=3,
        local_dir = "/content/drive/MyDrive/INM707/cartpole_v1", 
        config=config)


Trial name,status,loc,train_batch_size
objective_fn_CartPole-v0_31265_00000,RUNNING,172.28.0.2:730,32
objective_fn_CartPole-v0_31265_00001,PENDING,,64
objective_fn_CartPole-v0_31265_00002,PENDING,,32
objective_fn_CartPole-v0_31265_00003,PENDING,,64
objective_fn_CartPole-v0_31265_00004,PENDING,,32
objective_fn_CartPole-v0_31265_00005,PENDING,,64


(ImplicitFunc pid=730) 2022-04-22 11:20:23,241	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=730) 2022-04-22 11:20:23,278	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(objective_fn pid=730) 2022-04-22 11:20:23,278	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc,train_batch_size
objective_fn_CartPole-v0_31265_00000,RUNNING,172.28.0.2:730,32
objective_fn_CartPole-v0_31265_00001,PENDING,,64
objective_fn_CartPole-v0_31265_00002,PENDING,,32
objective_fn_CartPole-v0_31265_00003,PENDING,,64
objective_fn_CartPole-v0_31265_00004,PENDING,,32
objective_fn_CartPole-v0_31265_00005,PENDING,,64


Trial name,status,loc,train_batch_size
objective_fn_CartPole-v0_31265_00000,RUNNING,172.28.0.2:730,32
objective_fn_CartPole-v0_31265_00001,PENDING,,64
objective_fn_CartPole-v0_31265_00002,PENDING,,32
objective_fn_CartPole-v0_31265_00003,PENDING,,64
objective_fn_CartPole-v0_31265_00004,PENDING,,32
objective_fn_CartPole-v0_31265_00005,PENDING,,64


(objective_fn pid=730) 2022-04-22 11:20:35,420	INFO trainable.py:130 -- Trainable.setup took 12.143 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(objective_fn pid=730) 2022-04-22 11:20:35,420	WARNING util.py:55 -- Install gputil for GPU system monitoring.


...

Result for objective_fn_CartPole-v0_31265_00005:
  date: 2022-04-22_11-23-02
  done: false
  experiment_id: 8e5d6a4433154388bb0b0b947179691f
  hostname: 9b9552d30cc8
  iterations: 4
  iterations_since_restore: 5
  mean_reward: 24.57894736842105
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 51.9
    ram_util_percent: 32.4
  pid: 1044
  should_checkpoint: true
  time_since_restore: 13.265152931213379
  time_this_iter_s: 1.4505198001861572
  time_total_s: 13.265152931213379
  timestamp: 1650626582
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: '31265_00005'
  


Trial name,status,loc,train_batch_size,iter,total time (s),iterations,mean_reward
objective_fn_CartPole-v0_31265_00005,RUNNING,172.28.0.2:1044,64,6,14.7271,5,24.5789
objective_fn_CartPole-v0_31265_00000,TERMINATED,172.28.0.2:730,32,10,28.091,9,27.0795
objective_fn_CartPole-v0_31265_00001,TERMINATED,172.28.0.2:729,64,10,20.5998,9,27.5238
objective_fn_CartPole-v0_31265_00002,TERMINATED,172.28.0.2:876,32,10,19.9932,9,21.64
objective_fn_CartPole-v0_31265_00003,TERMINATED,172.28.0.2:932,64,10,20.7395,9,23.6
objective_fn_CartPole-v0_31265_00004,TERMINATED,172.28.0.2:987,32,10,20.0546,9,27.069


Result for objective_fn_CartPole-v0_31265_00005:
  date: 2022-04-22_11-23-08
  done: false
  experiment_id: 8e5d6a4433154388bb0b0b947179691f
  hostname: 9b9552d30cc8
  iterations: 8
  iterations_since_restore: 9
  mean_reward: 26.987179487179485
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 55.75
    ram_util_percent: 32.4
  pid: 1044
  should_checkpoint: true
  time_since_restore: 19.128239393234253
  time_this_iter_s: 1.485896348953247
  time_total_s: 19.128239393234253
  timestamp: 1650626588
  timesteps_since_restore: 0
  training_iteration: 9
  trial_id: '31265_00005'
  
Result for objective_fn_CartPole-v0_31265_00005:
  date: 2022-04-22_11-23-10
  done: true
  experiment_id: 8e5d6a4433154388bb0b0b947179691f
  experiment_tag: 5_train_batch_size=64
  hostname: 9b9552d30cc8
  iterations: 9
  iterations_since_restore: 10
  mean_reward: 29.6
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 52.25
    ram_util_percent: 32.4
  pid: 1044
  should_checkpoint: true
  time_since_res

Trial name,status,loc,train_batch_size,iter,total time (s),iterations,mean_reward
objective_fn_CartPole-v0_31265_00000,TERMINATED,172.28.0.2:730,32,10,28.091,9,27.0795
objective_fn_CartPole-v0_31265_00001,TERMINATED,172.28.0.2:729,64,10,20.5998,9,27.5238
objective_fn_CartPole-v0_31265_00002,TERMINATED,172.28.0.2:876,32,10,19.9932,9,21.64
objective_fn_CartPole-v0_31265_00003,TERMINATED,172.28.0.2:932,64,10,20.7395,9,23.6
objective_fn_CartPole-v0_31265_00004,TERMINATED,172.28.0.2:987,32,10,20.0546,9,27.069
objective_fn_CartPole-v0_31265_00005,TERMINATED,172.28.0.2:1044,64,10,20.58,9,29.6


2022-04-22 11:23:10,661	INFO tune.py:639 -- Total run time: 171.64 seconds (170.87 seconds for the tuning loop).


In [12]:
analysis.best_checkpoint.local_path

'/content/drive/MyDrive/INM707/cartpole_v1/objective_fn_2022-04-22_11-20-19/objective_fn_CartPole-v0_31265_00005_5_train_batch_size=64_2022-04-22_11-22-17/checkpoint_000009/'

## Save best params in a file

In [13]:
#save best config to file 
import json 
#path = '/content/drive/MyDrive/INM707/cartpole_v1/objective_fn_2022-04-21_20-53-06'

path = ''
for val in analysis.best_checkpoint.local_path.split('/')[1:-3]: 
  path = path + '/' + val 

fname = 'best_params.json'
full_path = path + '/' + fname 


f = open(full_path, "w")
json_dict = json.dumps(str(analysis.best_config))
f.write(json_dict)
f.close()


In [14]:
model_train_config = analysis.best_config

#Use hyper parameters to train model 

#Load best params to config

In [15]:
"""
import json
fname = '/content/drive/MyDrive/INM707/cartpole_v1/objective_fn_2022-04-21_20-53-06' + '/' + 'best_params.json'
print(fname)
with open(fname) as json_file:
    config_dict = json.load(json_file)

type(config_dict)
print(config_dict[:74])
temp = config_dict[:74] + '}'
temp
Dict = eval(temp)
Dict
"""

"\nimport json\nfname = '/content/drive/MyDrive/INM707/cartpole_v1/objective_fn_2022-04-21_20-53-06' + '/' + 'best_params.json'\nprint(fname)\nwith open(fname) as json_file:\n    config_dict = json.load(json_file)\n\ntype(config_dict)\nprint(config_dict[:74])\ntemp = config_dict[:74] + '}'\ntemp\nDict = eval(temp)\nDict\n"

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

device:  cuda:0


In [17]:
ray.shutdown()
ray.init(num_cpus= 2, num_gpus=1)

{'address': '172.28.0.2:65139',
 'gcs_address': '172.28.0.2:65139',
 'metrics_export_port': 45907,
 'node_id': '01ad1c272f3b12502817e9b0eecfe816da5503d5e15135c85d8756c3',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-04-22_11-23-45_238073_72/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-04-22_11-23-45_238073_72/sockets/raylet',
 'redis_address': None,
 'session_dir': '/tmp/ray/session_2022-04-22_11-23-45_238073_72',
 'webui_url': None}

## Train model on best params

In [ ]:
config = sac.DEFAULT_CONFIG.copy()

config["framework"] = "torch"
config["train_batch_size"] = 32 #tune.grid_search([32, 64])
config["target_network_update_freq"] = 32 #tune.grid_search([16, 32])
config["env"] = 'CartPole-v0' 
config["gamma"] = 0.95 #tune.uniform(0, 1)

#used from tuned values : https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/sac/cartpole-sac.yaml
config["tau"] = 1
config["optimization"] = {
        "actor_learning_rate": 0.005,
        "critic_learning_rate": 0.005,
        "entropy_learning_rate": 0.0001}
config["no_done_at_end"] = False


config["num_gpus"] = 1

model_train_config = config

In [19]:
model_train = tune.run("SAC",
                    stop={"episode_reward_mean": 100},
                    config = model_train_config,
                    local_dir = "/content/drive/MyDrive/INM707/cartpole_v1",
                    checkpoint_freq = 10)

Trial name,status,loc
SAC_CartPole-v0_64065_00000,PENDING,


(SACTrainer pid=1191) 2022-04-22 11:28:57,649	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(SACTrainer pid=1191) 2022-04-22 11:28:57,650	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
SAC_CartPole-v0_64065_00000,RUNNING,172.28.0.2:1191


(SACTrainer pid=1191) 2022-04-22 11:29:01,684	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Trial name,status,loc
SAC_CartPole-v0_64065_00000,RUNNING,172.28.0.2:1191


...

Result for SAC_CartPole-v0_64065_00000:
  agent_timesteps_total: 10700
  custom_metrics: {}
  date: 2022-04-22_11-31-19
  done: false
  episode_len_mean: 96.09
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 96.09
  episode_reward_min: 11.0
  episodes_this_iter: 0
  episodes_total: 145
  experiment_id: b1f7de8926e041bea0e209f890048691
  hostname: 9b9552d30cc8
  info:
    last_target_update_ts: 10674
    learner:
      default_policy:
        learner_stats:
          actor_loss: -43.51719284057617
          alpha_loss: -0.0368126705288887
          alpha_value: 2.3763272762298584
          critic_loss: 0.0766201913356781
          log_alpha_value: 0.8655561804771423
          max_q: 44.65904235839844
          mean_q: 41.723411560058594
          min_q: 25.230871200561523
          policy_t: 0.5
          target_entropy: 0.6792842149734497
        mean_td_error: 0.3284885883331299
        td_error:
        - 0.40296173095703125
        - 0.4847278594970703
       

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
SAC_CartPole-v0_64065_00000,RUNNING,172.28.0.2:1191,95,137.836,10900,97.82,200,11,97.82


Result for SAC_CartPole-v0_64065_00000:
  agent_timesteps_total: 11100
  custom_metrics: {}
  date: 2022-04-22_11-31-25
  done: false
  episode_len_mean: 99.7
  episode_media: {}
  episode_reward_max: 200.0
  episode_reward_mean: 99.7
  episode_reward_min: 11.0
  episodes_this_iter: 0
  episodes_total: 147
  experiment_id: b1f7de8926e041bea0e209f890048691
  hostname: 9b9552d30cc8
  info:
    last_target_update_ts: 11070
    learner:
      default_policy:
        learner_stats:
          actor_loss: -42.99762725830078
          alpha_loss: -0.0908115953207016
          alpha_value: 2.4699864387512207
          critic_loss: 0.09571187198162079
          log_alpha_value: 0.9042126536369324
          max_q: 46.209293365478516
          mean_q: 40.43885803222656
          min_q: 0.9439122676849365
          policy_t: 0.5
          target_entropy: 0.6792842149734497
        mean_td_error: 0.29957807064056396
        td_error:
        - 0.493255615234375
        - 0.2512226104736328
        -

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
SAC_CartPole-v0_64065_00000,TERMINATED,172.28.0.2:1191,98,142.222,11200,101.55,200,11,101.55


2022-04-22 11:31:27,686	INFO tune.py:639 -- Total run time: 153.79 seconds (153.30 seconds for the tuning loop).


# Load trained model 

In [20]:
checkpoint_file = "/content/drive/MyDrive/INM707/cartpole_v1/SAC/SAC_CartPole-v0_64065_00000_0_2022-04-22_11-28-54/checkpoint_000090/checkpoint-90"
# "/content/drive/MyDrive/INM707/cartpole_v1/SAC/SAC_CartPole-v0_4e787_00000_0_2022-04-21_22-49-31/checkpoint_000270/checkpoint-270"
agent = sac.SACTrainer(config=model_train_config)
agent.restore(checkpoint_file)

2022-04-22 11:33:44,340	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-04-22 11:33:44,342	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-04-22 11:33:51,553	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-04-22 11:33:51,591	INFO trainable.py:496 -- Restored on 172.28.0.2 from checkpoint: /content/drive/MyDrive/INM707/cartpole_v1/SAC/SAC_CartPole-v0_64065_00000_0_2022-04-22_11-28-54/checkpoint_000090/checkpoint-90
2022-04-22 11:33:51,593	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 90, '_timesteps_total': 5760, '_time_total': 130.7810823917389, '_episodes_total': 144}


In [ ]:
import gym
done = False

env = wrap_env(gym.make('CartPole-v0'))
state = env.reset()
total_reward = 0 

while not done:
    action = agent.compute_single_action(state)
    action = env.action_space.sample() # take a random action

    state, reward, done, _ = env.step(action) # take a random action
    total_reward += reward

    #env.render()


env.close()
show_video()

In [ ]:
print("Total reward:", total_reward)